Expected Behind Model - Model Tuning and Building - GBM

In [1]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import set_shots_file_path
# from expected_score_model.domain.preprocessing.data_preprocessor import DataPreprocessor
# from expected_score_model.domain.contracts.mappings import Mappings
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Model Version

In [3]:
model_version = 6
model_name = 'expected_behind_set'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [4]:
RESPONSE = ModellingDataContract.RESPONSE_BEHIND

In [5]:
FEATURES = ModellingDataContract.feature_list_set_behind

In [6]:
MONOTONE_CONSTRAINTS = ModellingDataContract.monotone_constraints_set_behind

Load Data

In [7]:
df_modelling = pd.read_csv(set_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,GoalTrainingSet,GoalTestSet,GoalValidationSet,BehindTrainingSet,BehindTestSet,BehindValidationSet,MissTrainingSet,MissTestSet,MissValidationSet
11052,208,possGain,behind,1663.0,4,634,683.0,Geelong,Geelong,Tom Hawkins,Tom_Hawkins,Kick,41.0,25.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,0,1,1,0,105.0,32.0,73.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,41.0,48.0,48.0,26.0,25.0,21.0,21.0,14.0,683.0,644.0,642.0,641.0,49.0,39.0,8.062258,39.0,25.0,46.324939,0.570040,32.660913,0.116867,6.695990,0.364585,20.889203,1521.0,3.663562,True,True,False,False,True,False,False,True,False,False
11053,212,kickIn,goal,1730.0,4,787,885.0,Geelong,Geelong,Patrick Dangerfield,Patrick_Dangerfield,Kick,51.0,43.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,107.0,33.0,74.0,Kick,Contested Mark,Contest Target,Kick Inside 50 Result,51.0,53.0,53.0,53.0,43.0,38.0,38.0,38.0,885.0,852.0,851.0,850.0,98.0,33.0,5.385165,29.0,43.0,51.865210,0.977446,56.003541,0.069260,3.968302,0.214332,12.280314,841.0,3.367296,True,False,False,True,True,False,False,True,False,False
11054,213,centreBounce,goal,1754.0,4,956,979.0,Sydney,Sydney,Paddy McCartin,Paddy_McCartin,Kick,66.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,119.0,33.0,86.0,Kick,Free For,Kick Inside 50 Result,Kick Into F50,66.0,74.0,74.0,27.0,0.0,0.0,0.0,17.0,979.0,977.0,968.0,967.0,23.0,2.0,8.000000,14.0,0.0,14.000000,0.000000,0.000000,0.482343,27.636207,2.588598,148.315731,196.0,2.639057,True,True,False,False,True,False,False,False,True,False
11055,219,centreBounce,goal,1846.0,4,1252,1266.0,Sydney,Sydney,Chad Warner,Chad_Warner,Kick,26.0,3.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,125.0,39.0,86.0,Kick,Uncontested Mark,Ground Kick,Handball,26.0,18.0,-5.0,-2.0,3.0,2.0,-4.0,3.0,1266.0,1265.0,1264.0,1260.0,14.0,1.0,8.062258,54.0,3.0,54.083269,0.055499,3.179830,0.118569,6.793500,0.365993,20.969862,2916.0,3.988984,True,True,False,False,True,False,False,True,False,False
11056,228,centreBounce,goal,1912.0,4,1519,1560.0,Geelong,Geelong,Jeremy Cameron,Jeremy_Cameron,Kick,41.0,21.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,131.0,45.0,86.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,41.0,49.0,49.0,25.0,21.0,17.0,17.0,33.0,1560.0,1535.0,1532.0,1531.0,41.0,25.0,8.944272,39.0,21.0,44.294469,0.493941,28.300756,0.127885,7.327246,0.400462,22.944787,1521.0,3.663562,True,True,False,False,True,False,False,True,False,False


In [8]:
df_modelling[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(df_modelling['Initial_State'])

In [9]:
training_data = df_modelling[df_modelling[RESPONSE+"TrainingSet"]]
test_data = df_modelling[df_modelling[RESPONSE+"TestSet"]]
cal_data = df_modelling[df_modelling[RESPONSE+"ValidationSet"]]

In [10]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]
X_cal, y_cal = cal_data.drop(columns=[RESPONSE]), cal_data[RESPONSE]

In [11]:
y_train.mean(), y_test.mean(), y_cal.mean()

(0.33352176370830977, 0.3336347197106691, 0.33352176370830977)

In [12]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,GoalTrainingSet,GoalTestSet,GoalValidationSet,BehindTrainingSet,BehindTestSet,BehindValidationSet,MissTrainingSet,MissTestSet,MissValidationSet,ballUp,centreBounce,kickIn,possGain,throwIn
9,101,possGain,goal,929.0,2,1504,1576.0,Brisbane Lions,Brisbane Lions,Oscar McInerney,Oscar_McInerney,Kick,59.0,-6.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,6,0,44.0,46.0,-2.0,Kick,Contested Mark,Kick,Mark On Lead,59.0,70.0,44.0,48.0,-6.0,-4.0,39.0,33.0,1576.0,1542.0,1540.0,1508.0,72.0,34.0,11.180340,19.0,6.0,19.924859,0.305879,17.525568,0.314407,18.014187,1.196693,68.565478,361.0,2.944439,True,True,False,False,False,True,False,True,False,False,0,0,0,1,0
10,108,possGain,goal,982.0,2,1782,1859.0,Sydney,Sydney,Sam Reid,Sam_Reid,Kick,40.0,36.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,50.0,46.0,4.0,Kick,Uncontested Mark,Kick,Uncontested Mark,40.0,44.0,43.0,50.0,36.0,32.0,48.0,40.0,1859.0,1835.0,1833.0,1814.0,77.0,24.0,5.656854,38.0,36.0,52.345009,0.758378,43.451842,0.089092,5.104600,0.275545,15.787586,1444.0,3.637586,True,True,False,False,False,True,False,False,True,False,0,0,0,1,0
11,122,possGain,goal,1074.0,3,28,72.0,Sydney,Sydney,Nick Blakey,Nick_Blakey,Kick,28.0,-14.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,51.0,52.0,-1.0,Kick,Uncontested Mark,Kick Inside 50 Result,Kick Into F50,28.0,35.0,35.0,32.0,-14.0,-12.0,-12.0,35.0,72.0,38.0,36.0,36.0,44.0,34.0,7.280110,50.0,14.0,51.923020,0.273009,15.642246,0.119147,6.826615,0.368686,21.124166,2500.0,3.912023,True,True,False,False,False,True,False,True,False,False,0,0,0,1,0
15,144,throwIn,goal,1283.0,3,1014,1041.0,Sydney,Sydney,Callum Mills,Callum_Mills,Kick,29.0,-3.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,58.0,71.0,-13.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,29.0,37.0,37.0,22.0,-3.0,-2.0,-2.0,-16.0,1041.0,1019.0,1018.0,1017.0,27.0,22.0,8.062258,49.0,3.0,49.091751,0.061148,3.503532,0.130680,7.487397,0.405895,23.256079,2401.0,3.891820,True,True,False,False,False,True,False,True,False,False,0,0,0,0,1
18,173,possGain,goal,1521.0,3,2013,2063.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,51.0,13.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,65.0,101.0,-36.0,Kick,Uncontested Mark,Kick,Uncontested Mark,51.0,57.0,32.0,36.0,13.0,8.0,-12.0,-12.0,2063.0,2024.0,2023.0,2020.0,50.0,39.0,7.810250,27.0,13.0,29.966648,0.448723,25.709954,0.194647,11.152463,0.643304,36.858597,729.0,3.295837,True,False,False,True,False,True,False,True,False,False,0,0,0,1,0


Preprocess Data

In [13]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [14]:
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]
X_cal_preproc = X_cal[FEATURES]

In [15]:
X_train_preproc.shape, X_test_preproc.shape

((7076, 5), (2212, 5))

In [16]:
X_train_preproc.head()

,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle
0,46.0,-41.0,52.009614,0.908067,0.075999
1,52.0,35.0,43.600459,0.931882,0.088007
3,59.0,27.0,33.015148,0.957589,0.112618
4,33.0,12.0,46.572524,0.260602,0.133410
5,52.0,36.0,44.407207,0.945311,0.084822


In [17]:
X_test_preproc.head()

,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle
9,59.0,-6.0,19.924859,0.305879,0.314407
10,40.0,36.0,52.345009,0.758378,0.089092
11,28.0,-14.0,51.923020,0.273009,0.119147
15,29.0,-3.0,49.091751,0.061148,0.130680
18,51.0,13.0,29.966648,0.448723,0.194647


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [18]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints=MONOTONE_CONSTRAINTS)

In [19]:
xgb_tuner.tune_hyperparameters()

[I 2023-07-11 15:34:49,903] A new study created in memory with name: no-name-0fd8db58-cf29-43fe-b71d-29bcca3c5f31
[I 2023-07-11 15:34:49,937] Trial 0 finished with value: 0.6178359840125964 and parameters: {'max_depth': 10, 'min_child_weight': 8, 'eta': 0.47444986027445185, 'gamma': 0.01584969481059091, 'lambda': 1.067419107155443, 'alpha': 0.07697615149021918, 'subsample': 0.708036324203885, 'colsample_bytree': 0.4616596016962103}. Best is trial 0 with value: 0.6178359840125964.
[I 2023-07-11 15:34:49,965] Trial 1 finished with value: 0.6390009125966374 and parameters: {'max_depth': 14, 'min_child_weight': 26, 'eta': 0.06510541942255398, 'gamma': 0.7639033466077108, 'lambda': 0.009629379085605795, 'alpha': 0.00010436458294189452, 'subsample': 0.22007935584302687, 'colsample_bytree': 0.8785172978371889}. Best is trial 0 with value: 0.6178359840125964.
[I 2023-07-11 15:34:49,996] Trial 2 finished with value: 0.6657919425082829 and parameters: {'max_depth': 16, 'min_child_weight': 23, 'e

Number of finished trials:  1000
Best trial:
  Value: 0.5877047569297693
  Params: 
    max_depth: 6
    min_child_weight: 24
    eta: 0.30662681044857415
    gamma: 1.4873515608096657
    lambda: 0.0007330287667095376
    alpha: 0.007780282745973114
    subsample: 0.48000817424088255
    colsample_bytree: 0.8079822620295684


In [20]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 6,
 'min_child_weight': 24,
 'eta': 0.30662681044857415,
 'gamma': 1.4873515608096657,
 'lambda': 0.0007330287667095376,
 'alpha': 0.007780282745973114,
 'subsample': 0.48000817424088255,
 'colsample_bytree': 0.8079822620295684}

Training Model - SuperXGBClassifier class for training and predictions

In [21]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = MONOTONE_CONSTRAINTS

In [22]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test,
                               X_cal = X_cal_preproc,
                               y_cal = y_cal,
                               params = params)

In [23]:
super_xgb.fit()

[0]	validation_0-logloss:0.65247	validation_1-logloss:0.65112
[1]	validation_0-logloss:0.63257	validation_1-logloss:0.63067
[2]	validation_0-logloss:0.62034	validation_1-logloss:0.61893
[3]	validation_0-logloss:0.61375	validation_1-logloss:0.61202
[4]	validation_0-logloss:0.60950	validation_1-logloss:0.60831
[5]	validation_0-logloss:0.60666	validation_1-logloss:0.60599
[6]	validation_0-logloss:0.60458	validation_1-logloss:0.60546
[7]	validation_0-logloss:0.60300	validation_1-logloss:0.60397
[8]	validation_0-logloss:0.60185	validation_1-logloss:0.60341
[9]	validation_0-logloss:0.60092	validation_1-logloss:0.60289
[10]	validation_0-logloss:0.60035	validation_1-logloss:0.60167
[11]	validation_0-logloss:0.60022	validation_1-logloss:0.60219
[12]	validation_0-logloss:0.59945	validation_1-logloss:0.60314
[13]	validation_0-logloss:0.59913	validation_1-logloss:0.60211
[14]	validation_0-logloss:0.59894	validation_1-logloss:0.60254
[15]	validation_0-logloss:0.59848	validation_1-logloss:0.60186
[1

In [24]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8079822620295684, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1.4873515608096657, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.30662681044857415, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=24, missing=nan,
              monotone_constraints={'Angle_to_Middle_Goal': 1,
                                    'Distance_to_Middle_Goal': 1,
                                    'Visible_Goal_Angle': -1, 'x0': 1},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [25]:
super_xgb.xgb_model.get_booster().feature_names

['x0',
 'y0',
 'Distance_to_Middle_Goal',
 'Angle_to_Middle_Goal',
 'Visible_Goal_Angle']

In [26]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [27]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]
cal_probas = super_xgb.predict_proba(X_cal_preproc)[:, 1]

In [28]:
super_xgb.calibrate()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [29]:
train_cal_probas = super_xgb.predict_proba(X_train_preproc, calibrate=True)
test_cal_probas = super_xgb.predict_proba(X_test_preproc, calibrate=True)
cal_cal_probas = super_xgb.predict_proba(X_cal_preproc, calibrate=True)

Check Average Predictions

In [30]:
train_probas.mean(), training_data[RESPONSE].mean(), train_cal_probas.mean()

(0.33690697, 0.33352176370830977, 0.33361522391218157)

In [31]:
test_probas.mean(), test_data[RESPONSE].mean(), test_cal_probas.mean()

(0.33624902, 0.3336347197106691, 0.33291161276393577)

In [32]:
cal_probas.mean(), cal_data[RESPONSE].mean(), cal_cal_probas.mean()

(0.3367443, 0.33352176370830977, 0.333521760241809)

Export model

In [33]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [34]:
train_info = training_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export['xgb_probas_cal'] = train_cal_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Order,Behind,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,17,183.0,1,46.0,-41.0,52.009614,0.908067,0.075999,0,0.459242,0.454404
1,202101_BrisbaneLions_Sydney,28,295.0,1,52.0,35.0,43.600459,0.931882,0.088007,0,0.372897,0.375882
2,202101_BrisbaneLions_Sydney,53,537.0,1,59.0,27.0,33.015148,0.957589,0.112618,0,0.380144,0.382936
3,202101_BrisbaneLions_Sydney,59,578.0,0,33.0,12.0,46.572524,0.260602,0.133410,0,0.352027,0.355097
4,202101_BrisbaneLions_Sydney,62,621.0,0,52.0,36.0,44.407207,0.945311,0.084822,0,0.355224,0.358327


In [35]:
test_info = test_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export['xgb_probas_cal'] = test_cal_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Order,Behind,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,101,929.0,0,59.0,-6.0,19.924859,0.305879,0.314407,0,0.049156,0.025064
1,202101_BrisbaneLions_Sydney,108,982.0,0,40.0,36.0,52.345009,0.758378,0.089092,0,0.355224,0.358327
2,202101_BrisbaneLions_Sydney,122,1074.0,0,28.0,-14.0,51.923020,0.273009,0.119147,0,0.419161,0.419450
3,202101_BrisbaneLions_Sydney,144,1283.0,0,29.0,-3.0,49.091751,0.061148,0.130680,0,0.401249,0.402994
4,202101_BrisbaneLions_Sydney,173,1521.0,0,51.0,13.0,29.966648,0.448723,0.194647,0,0.268685,0.265571


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")